In [ ]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key
import calendar

Token = 'tylRGHbxqiZUDNymMowCBFCuEqeULiWk'
Token_NOAA=Token
API_Token='Token d8f94b93417333267ca0edcca7dc14a9e035e1bb'

In [ ]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

# For Loop with several zip codes since 1985 to 2015


In [ ]:
bigdf = pd.DataFrame()
states = {
    'Napa': {
        'zip': '94558',
        'years': ['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
        },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
#         },
    'Columbia': {
        'zip': '98813',
        'years':['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
        },
    'Sonoma': {
        'zip': '95476',
        'years': ['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
        },
    'Santa': {
        'zip': '95062',
        'years': ['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
        },
    'Yakima': {
        'zip': '98903',
        'years':['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
        },
    'Dundee': {
        'zip': '97045',
        'years': ['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
        },
    'Willamette': {
        'zip': '00077',
        'years': ['1985','1986','1987','1988','1989','1990', '1991','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
        }
    #etc...
}

In [ ]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        #r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=ZIP:94558&datatypeid=TMIN&sortfield=name&sortorder=desc&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        #p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=GHCND&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #e = json.loads(p.text)
        items_MAX = [item for item in d['results'] if item['datatype']=='PRCP']
#         items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        # #get the date field from all average temperature readings
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
#         dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        df_test['date'] = dates_temp_MAX
        df_test['PRCP'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['PRCP'+state]] = item['value']
#         df_test['avgMinTemp'+state] = np.nan
#         for item in items_MIN:
#             date = item['date'].split('T')[0]
#             df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
        small_df = pd.concat([small_df,df_test])
    if bigdf.empty:
        bigdf = small_df
    else:
        bigdf = pd.merge(bigdf,small_df)
bigdf.to_csv( 'Prcp_Bigdf.csv', index = False)

In [ ]:
bigdf

In [ ]:
#dataframe save as csv file
#bigdf.to_csv( 'weatherapi.csv', index = False)

In [ ]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if walladf.empty:
        walladf = small_df
    else:
        walladf = pd.merge(walladf,small_df)
walladf.head(20)